# Welcome to OpenAI GPT v2.5

**NOTE: This notebook is the continuation of <a href="https://github.com/AvishakeAdhikary/Neural-Networks-From-Scratch/blob/main/GPT%20from%20Scratch.ipynb">GPT from Scratch</a> and <a href="https://github.com/AvishakeAdhikary/Neural-Networks-From-Scratch/blob/main/GPT%20Tokenizer.ipynb">GPT Tokenizer</a> notebooks.**

In this notebook we are going to reproduce the <a href="https://github.com/openai/gpt-2">OpenAI's GPT 2</a> model, the (`124M`) version of it.

Now, when OpenAI released GPT 2, they released it with this <a href="https://openai.com/index/better-language-models/">blog post</a> and this <a href="https://d4mucfpksywv.cloudfront.net/better-language-models/language_models_are_unsupervised_multitask_learners.pdf">paper</a>. And on top of that, they released this <a href="https://github.com/openai/gpt-2">code</a> on GitHub...

But, when reproducing GPT 2, we have to be careful, because we are going to be reproducing the `124M` parameter model. And the thing to be careful with it is there's always a sub-series of models of different sizes when these model releases are made and usually the biggest model is called the **"GPT"**.

Let's consider the charts that we have in the paper for a second: \
![OpenAIGPT2 Graphs](ExplanationMedia/Images/OpenAIGPT2Graphs.png)

Now the reason we have multiple models is because, according to the above graphs we see that we consider the `Number of parameters in the Language Model` in the `x-axis` and the `y-axis` we put a lot of *downstream metrics* that we are interested in like ("Translation", "Summerization", "Question Answering") and so on and we can chart out the *downstream metrics* as the model size increases.

And in the paper we see a table like this:

| Parameters | Layers | $d_{model}$ |
|------------|--------|-----------|
| 117M       | 12     | 768       |
| 345M       | 24     | 1024      |
| 762M       | 36     | 1280      |
| 1542M      | 48     | 1600      |

And we see `4` models in the `GPT-2` sub series, starting at `124M` all the way up to `1558M`...

But you might be thinking that I might have made a mistake because, in the table the numbers are different and the numbers I spoke of are different. And the reason my numbers disagree with this table is because this entire table is wrong and if we go to their <a href="https://github.com/openai/gpt-2">GitHub repository</a> we see a note that says:
> * *Note that our original parameter counts were wrong due to an error (in our previous blog posts and paper). Thus you may have seen small referred to as 117M and medium referred to as 345M.*

And in the `124M` parameter model, we see that they used `12 Layers` in the Transformer and `768 Channel Dimensions` in the Transformer.

And by the end of this notebook we will try to beat the original `GPT-2 124M` model and will be looking at loss graphs to see our model perform better.

The thing to note here is, this paper is more than 5 years old now and it was probably a very complicated optimization at the time and the computation was very low at the time, but today we can reproduce the same model's performance in roughly an hour or so and it will cost us around $10 (if we want to do this on a cloud compute, or in other words, a computer that we can all rent). 

And one more thing to mention is, OpenAI did release it's model's weights and it is available on it's GitHub repository, but it's paper is not good with all of it's details with the training.

So, in addition to the GPT-2 paper, we will also be referring to the <a href="https://arxiv.org/abs/2005.14165">GPT-3 paper</a>, which is a lot more concrete and a lot of the hyper-parameters and optimization settings and so on, which is not a huge departure from the architecture of GPT-2 version of the model.


So, let's do this...

# Understanding Hugging Face Pre-Trained Model

So, the first thing we'd like to do is start at the very end. Or in other words, we'll load the `GPT-2 124M` model as it was released by OpenAI and take it for a spin and sample some `tokens` from it.

Now, the issue is...

When we look at the code base and look for the <a href="">`model.py`</a> we see these imports:
```python
import numpy as np
import tensorflow as tf
from tensorflow.contrib.training import HParams
```

And we realise that the code is written in <a href="https://www.tensorflow.org/">TensorFlow</a> (another alternative for creating and training deep learning models offered by Google). Meaning that the original `GPT-2` code was written in TensorFlow and is not used anymore...

And as per our previous notebooks, we'd like to use PyTorch. And it will be a lot easier if we'd be able to work with the old explanations.

But the problem with that is that the initial code is in TensorFlow and we'd like to use PyTorch. So, in order to get the targets we'd like to use the <a href="https://huggingface.co/docs/transformers/en/index">`Hugging Face Transformers Library`</a> released at PyPi. We can use this <a href="https://huggingface.co/docs/transformers/en/installation">installation documentaiton</a> to walk through the steps to install the library in our system...

We can also check out Hugging Face's implementation of that transformer in their <a href="https://github.com/huggingface/transformers/blob/main/src/transformers/models/gpt2/modeling_gpt2.py">`modeling_gpt2.py`</a>. Which did a lot of work to convert all those TensorFlow code to PyTorch such that it becomes easier to load and work with.

So in particular we can look at the <a href="https://huggingface.co/openai-community/gpt2">Hugging Face GPT-2</a> model and load it using the Hugging Face transformers...


So this is what the code looks like:

```python
from transformers import GPT2LMHeadModel

huggingface_model = GPT2LMHeadModel.from_pretrained("gpt2")
huggingfaceStateDictionary = huggingface_model.state_dict()

for key, value in huggingfaceStateDictionary.items():
    print(key, value.shape)
```
Which gives us the result:
```python
transformer.wte.weight torch.Size([50257, 768])
transformer.wpe.weight torch.Size([1024, 768])
transformer.h.0.ln_1.weight torch.Size([768])
transformer.h.0.ln_1.bias torch.Size([768])
transformer.h.0.attn.c_attn.weight torch.Size([768, 2304])
transformer.h.0.attn.c_attn.bias torch.Size([2304])
transformer.h.0.attn.c_proj.weight torch.Size([768, 768])
transformer.h.0.attn.c_proj.bias torch.Size([768])
transformer.h.0.ln_2.weight torch.Size([768])
transformer.h.0.ln_2.bias torch.Size([768])
transformer.h.0.mlp.c_fc.weight torch.Size([768, 3072])
transformer.h.0.mlp.c_fc.bias torch.Size([3072])
transformer.h.0.mlp.c_proj.weight torch.Size([3072, 768])
transformer.h.0.mlp.c_proj.bias torch.Size([768])
transformer.h.1.ln_1.weight torch.Size([768])
transformer.h.1.ln_1.bias torch.Size([768])
transformer.h.1.attn.c_attn.weight torch.Size([768, 2304])
transformer.h.1.attn.c_attn.bias torch.Size([2304])
transformer.h.1.attn.c_proj.weight torch.Size([768, 768])
transformer.h.1.attn.c_proj.bias torch.Size([768])
transformer.h.1.ln_2.weight torch.Size([768])
transformer.h.1.ln_2.bias torch.Size([768])
transformer.h.1.mlp.c_fc.weight torch.Size([768, 3072])
transformer.h.1.mlp.c_fc.bias torch.Size([3072])
transformer.h.1.mlp.c_proj.weight torch.Size([3072, 768])
transformer.h.1.mlp.c_proj.bias torch.Size([768])
...
transformer.h.11.mlp.c_proj.bias torch.Size([768])
transformer.ln_f.weight torch.Size([768])
transformer.ln_f.bias torch.Size([768])
lm_head.weight torch.Size([50257, 768])
```

One awkward thing about this is, when we say `gpt2` it actually loads the `124M` parameter model and if we want the actual `GPT-2` model we'd specify it as `gpt2-xl`...

Now when we actually get this `GPT-2` initialized, we want to get the **state dictionary** which is the **raw tensors loaded with values** and we can get those using the `.state_dict()` method. and we can print the `key` (which are the tensors) and the `value` (which are the tensor values) and we can look at the shapes of the `value` tensors to get an idea of the shapes of the states in the model...

So, we can now look at the different parameters inside the `GPT-2` model and their shapes...

And we can see that there are a lot of short forms of the terms that we already know of, so let's recall that:
1. **wte**: Word Token Embeddings
2. **wpe**: Word Position Embeddings
3. **ln**: Layer Normalization
4. **attn**: Attention
5. **c_attn**: Cross Attention (awkward because `GPT-2` is a decoder only architecture and should be named **self attention**)
6. **c_proj**: Projection layer within attention or MLP
7. **mlp**: Multi-Layer Perceptron
8. **lm_head**: Language Model Head (output layer)
9. **c_fc**: Current/Common Fully Connected Layer

We initially can recall the very first key-value pair `transformer.wte.weight torch.Size([50257, 768])` as the `Word Token Embeddings` having a shape of `[50257, 768]` and it comes from the `50257` vocabulary of tokens (which is exactly the number of tokens we spoke about in our <a href="https://github.com/AvishakeAdhikary/Neural-Networks-From-Scratch/blob/main/GPT%20Tokenizer.ipynb">Tokenizer Notebook</a>) having `768` feature space (or embedding vector space, or `768 dimensional embedding`)...

We can also look at the second key-value pair `transformer.wpe.weight torch.Size([1024, 768])`, we can recall them as `Word Positional Embeddings` having a shape of `[1024, 768]`. So, because `GPT-2` has a maximum sequence length of `1024` we have upto `1024` positions that each token can attend to in the past. And every one of those positions in `GPT-2` has a fixed vector of `768` that is learnt by optimization.

And everything else is just the other weights and biases of this transformer...

So, now for example, if we take just the `Positional Embeddings` and we flatten it out (we get a `[1, 768]` vector) and take just the first `20` elements of the `768` embeddings we can see that we get the proper weights as an output for this code:

```python
huggingfaceStateDictionary['transformer.wpe.weight'].view(-1)[:20]
```
We get:
```python
tensor([-0.0188, -0.1974,  0.0040,  0.0113,  0.0638, -0.1050,  0.0369, -0.1680,
        -0.0491, -0.0565, -0.0025,  0.0135, -0.0042,  0.0151,  0.0166, -0.1381,
        -0.0063, -0.0461,  0.0267, -0.2042])
```

And we can plot these weights and try to see what they represent like this:

```python
import matplotlib.pyplot as plt
%matplotlib inline

plt.imshow(huggingfaceStateDictionary['transformer.wpe.weight'], cmap='gray')
```

![GPT-2.transformer.wpe.weight](ExplanationMedia/Images/GPT-2.transformer.wpe.weight.png)

And we can see that this has structure, because these positional embeddings end up learning these **sinusoids** and **cosines** to represent each of these positions and each row here stands in for that position and is processed by the transformer to recover all the relative positions and realise which token is where and attend to them depending on their position not just their content...

So now if we look at the individual columns of these we see:

```python
plt.plot(huggingfaceStateDictionary['transformer.wpe.weight'][:, 150])
plt.plot(huggingfaceStateDictionary['transformer.wpe.weight'][:, 200])
plt.plot(huggingfaceStateDictionary['transformer.wpe.weight'][:, 250])
```
![GPT-2Graphs.transformer.wpe.weight](ExplanationMedia/Images/GPT-2Graphs.transformer.wpe.weight.png)

So, we still don't know what these embeddings are doing and why they are the way they are.

But we can still see that the lines are a little noisy and jittery and that is because this model was not fully trained, and the more trained this model becomes the more we'd expect these graphs to smooth out, which also tells us that the original `GPT-2` is an **under-trained** model.

If I remember correctly, in the original "Attention-Is-All-You-Need" paper, the `positional embeddings` are actually initialized and fixed to sinusoids and cosines of different frequencies, but in `GPT-2` these are trained from scratch and they seem to recover these features during the optimization.

Now, using the `Hugging Face Transformers` we can not just get all the raw weights but also get something called `pipeline` and sample from it...

Here is the sample code snippet for `5` different generations of the same context window of tokens `"Hello, I'm a language model,"`:

```python
from transformers import pipeline, set_seed

set_seed(42)

generator = pipeline("text-generation", model="gpt2")
generator("Hello, I'm a language model,", max_length=50, num_return_sequences=5)
```
For which we get:
```python
[{'generated_text': "Hello, I'm a language model, but what I'm really doing is making a human-readable document. There are other languages, but those are the ones I like the most. To do your research, please contact me, this isn't your"},
 {'generated_text': "Hello, I'm a language model, not a syntax model. That's why I like it. I've done a lot of programming projects.\n\nBut my job as a C programmer is to sort through every single line of the script so I"},
 {'generated_text': "Hello, I'm a language model, and I'll do it in no time!\n\nOne of the things we learned from talking to my friend from college a bit earlier, and in the context of the current language model I think it's important"},
 {'generated_text': 'Hello, I\'m a language model, not a command line tool.\n\nIf my code is simple enough:\n\nif (use (string-replace "\\r" ))) {\n\nconsole. log\n\n}\n\nthat\'s'},
 {'generated_text': "Hello, I'm a language model, I've been using Language in all my work. Just a small example, let's see a simplified example. I'm making an API for a game where I want a character to play a little bit of a"}]
```

Sadly, even though we are setting a seed we get different generations from both the code and the official <a href="https://huggingface.co/openai-community/gpt2">Hugging Face GPT-2 Hosted Inference API</a>.

But at this stage what is important is, we are getting coherent text and we were successfully able to load the model and look at all of it's parameters and the keys tell us, where in the model these come from...

But we want to actually write our own `GPT-2` class so that we have a full understanding of what's happening there and we also don't want to work with something like the <a href="https://github.com/huggingface/transformers/blob/main/src/transformers/models/gpt2/modeling_gpt2.py">`modeling_gpt2.py`</a> because it's too complicated and we want to write this from scratch ourselves.

So we are going to be implementing our `GPT-2` model in `GPT_v2.5.py` script inside our `GPT Scripts` directory in parallel...

But first let's load the `GPT-2 124M` into our `GPT_v2.5.py` for the class that we are going to develop from scratch, which is going to give us confidence that we can load the OpenAI model and there's a setting of weights that exactly is the `124M` model and we will try to surpass our own created `GPT` class...

So, we're going to get different weights and everything is going to look different and hopefully even better and we will have the confidence that we are in the same model family and same model class and we just have to re-discover a good setting of the weights from scratch... 

So let's now write the `GPT-2` model and let's load the weights and make sure that we can also generate text that looks coherent...

# Hugging Face Pre-Trained Weight Transfer

## Transformer - Architecture

Let's now swing over to the <a href="https://arxiv.org/abs/1706.03762">"Attention Is All You Need" paper</a> that started everything and look at the Transformer architecture:

![Transformer_Model_Architecture](ExplanationMedia/Images/Transformer_Model_Architecture.png)

Now, once again, like the last notebook, we mentioned that this architecture has changed over the years and `GPT-2` is slightly modified than the original `Transformer`... 

In particular, we do **NOT** have the **Encoder**, and `GPT-2` is a **Decoder** only `Transformer` as we call it. In addition to that the **Cross-Attention** that is used by that **Encoder** is also **missing**. Everything else stays almost the same, but there are some differences that we are going to see next...

So, there are two main differences: \
When we go to the `GPT-2` <a href="https://d4mucfpksywv.cloudfront.net/better-language-models/language_models_are_unsupervised_multitask_learners.pdf">paper</a>, under section `2.3 Model` we see that there's a re-shuffling of the layer-normalizations (they change place) and an additional layer normalization was added after the final self-attention block...

## GPT-2 Skeleton

### Hyper-Parameters (`GPTConfiguration`)

Let's now implement the skeleton of the `nn.Module`(s) in our GPT Script and in particular we want to match up the schema that we got from `Hugging Face GPT-2`...

And we will use a decorator called `@dataclass` which provides a decorator and functions for automatically adding generated special methods such as `__init__()` and `__repr__()` to user-defined classes...

And we will use it to define all the hyper-parameters as a Class called `GPTConfiguration`...

Now because we are going to be implementing the `124M GPT-2 Model`, when we go to the paper we see these hyper-parameters:
1. block-size (context window) → 1024
2. vocabulary-size (token vocabulary) → 50257
3. n-layer (number of layers) → 12
4. n-head (number of self-attention heads) → 12
5. embedding-dimensions ($d_{model}$) → 768

So let's implement this now...

For now our code looks like this:
```python
from dataclasses import dataclass

# GPT configuration hyper-parameters
@dataclass
class GPTConfiguration:
    blockSize: int = 1024
    vocabularySize: int = 50257
    numberOfLayers: int = 12
    numberOfHeads: int = 12
    numberOfEmbeddingDimensions: int = 768
```

### `GPTModel`

Now we will be able to use this configuration under the `GPTModel` class that we are going to write...

For now our empty `GPTModel` class looks like this:
```python
import torch
import torch.nn.functional as F

# GPT model architecture
class GPTModel(torch.nn.Module):
    def __init__(self, configuration):
        super().__init__()
        self.configuration = configuration
```

Now we want to **copy** the schema from the **Hugging Face `GPT-2` model** by utilizing the `huggingfaceStateDictionary`...

And here's what I came up with...

We see that the container in the schema is called `transformer` which contains all the modules and we can create something like that using `torch.nn.ModuleDict` which is just a dictionary of torch `Module`(s) which let's us index into `Module`(s) using **keys**, just like a normal python dictionary...

Within that we can create something called `wordTokenEmbeddings` which corresponds with `wte` and create something called `wordPositionalEmbeddings` which corresponds with `wpe`, and we can match the shapes and create our initial layers...

Then in the **Hugging Face `GPT-2` model** we see that we have a long list of **hidden** layers represented by a `.h` and followed by a range of number `.0` to `.11` hinting us about the number of layers as `12`, so we can now utilize our `numberOfLayers` hyper-parameter to construct these long list of layers. And instead of a `torch.nn.ModuleDict` we can use a `torch.nn.ModuleList` instead, which is just a list of `Module`(s).

The important thing to note is, in those hidden layers we see different kinds of layer **weights** and **biases** of different **layers** all having their own shapes and sizes, but we do see a pattern that they repeat themselves in terms of **layer number**, so for now we can just consider these **layers** as `Block`(s) and iterate them through a list and return itself to the list. Keep in mind that the `Block`'s defination has not been defined yet, and we will define it later, but we want all the `Block`(s) to take in the same `configuration` object and construct the layer objects through it, because we already have all the hyper-parameters set inside it...

Now that we have our long list of **hidden layers** it is time to construct the final **layer normalization** layer according to the `GPT-2` paper, so we can create something like `finalLayerNorm` and match the shapes which corresponds to the `ln_f`...

And lastly we can construct our **final classifier** (or the **language model head**) which is just a **Linear Layer** that projects all the **embeddings** to their respective **tokens**, having **no bias**. So, we can easily construct this **languageModelingHead** which corresponds to the `lm_head` and finish with our skeleton of the `GPT-2` model...

Now we end up with a code like this:

```python
from dataclasses import dataclass
import torch
import torch.nn.functional as F


# GPT configuration hyper-parameters
@dataclass
class GPTConfiguration:
    blockSize: int = 1024
    vocabularySize: int = 50257
    numberOfLayers: int = 12
    numberOfHeads: int = 12
    numberOfEmbeddingDimensions: int = 768

# GPT model architecture
class GPTModel(torch.nn.Module):
    def __init__(self, configuration):
        super().__init__()
        self.configuration = configuration

        self.transformer = torch.nn.ModuleDict(dict(
            wordTokenEmbeddings = torch.nn.Embedding(configuration.vocabularySize, configuration.numberOfEmbeddingDimensions),
            wordPositionalEmbeddings = torch.nn.Embedding(configuration.blockSize, configuration.numberOfEmbeddingDimensions),
            hidden = torch.nn.ModuleList(Block(configuration) for _ in range(configuration.numberOfLayers)),
            finalLayerNorm = torch.nn.LayerNorm(configuration.numberOfEmbeddingDimensions)
        ))

        self.languageModelingHead = torch.nn.Linear(configuration.numberOfEmbeddingDimensions, configuration.vocabularySize, bias=False)

model = GPTModel(GPTConfiguration())
```

### `Block`(s)

Let's now create the `Block` class which is currently undefined...

Now, here the `Block` refers to the `Transformer Block` that gets repeated again and again as hidden layers...

Now, according to our <a href="https://github.com/AvishakeAdhikary/Neural-Networks-From-Scratch/blob/main/GPT%20from%20Scratch.ipynb">GPT from Scratch</a> notebook, we already defined this `Block`, and as we mentioned `GPT-2` also has a slightly modified `Transformer Block`...

For now we are going to use this template:
```python
# Transformer Block
class Block(torch.nn.Module):
    def __init__(self, configuration):
        super().__init__()
    
    def forward(self, inputs):
        pass
```

For, now we understand that we are left with the following modules & properties:
1. Add & Norm
2. Attention
3. Feed Forward Network
4. Residual Pathways

Let's start with **Addition and Normalization (Add & Norm)**.

According to the diagram, the **Add & Norm** is there **AFTER** the **Attention & Feed Forward Network**, but in our case we will use them **BEFORE** the **Attention & Feed Forward Network**, making it a **Pre-Normalization (Pre-Norm)**...

Then we have our **Attention** module, and for now we can relate it to the attention we built in our last <a href="https://github.com/AvishakeAdhikary/Neural-Networks-From-Scratch/blob/main/GPT%20from%20Scratch.ipynb">GPT from Scratch</a> notebook. Specifically, we built two modules (`MultiHeadAttention` and `Head`), but here we will implement both modules in a combined and mathematically optimized class called `CausalSelfAttention`...

Then we have our **Feed Forward Network**, and we will call it our **MultiLayerPerceptron**.

The thing to note is, once again two of the modules (`CausalSelfAttention` and `MultiLayerPerceptron`) remain undefined, and we are going to define them later...

For now, we end up with a code like this:
```python
# Transformer Block
class Block(torch.nn.Module):
    def __init__(self, configuration):
        super().__init__()
        self.layerNormalization1 = torch.nn.LayerNorm(configuration.numberOfEmbeddingDimensions)
        self.attention = CausalSelfAttention(configuration)
        self.layerNormalization2 = torch.nn.LayerNorm(configuration.numberOfEmbeddingDimensions)
        self.multiLayerPerceptron = MultiLayerPerceptron(configuration)
    
    def forward(self, inputs):
        pass
```

And lastly, we arrive at the **Residual Pathways**, and we see that the normalizations are **inside** the residual stream, or in other words, the residual pathway has normalizations **inside** them (which is not very good or desirable from an optimization perspective) and we actually prefer to have a single and clean residual stream all the way from **supervision** to all the way to the **inputs (or `tokens`)**, which is desirable because the gradients that flow from the top distributes the gradients equally because of additions, indicating that the gradients from the top flow straight to the inputs through the residual pathway (unchanged) but then addition to that, the gradient also flows through the blocks and the blocks contribute their own contribution over time when the optimization kicks in.

Which means that we want to apply a **clean residual pathway**. And to do that we need the normalization to be applied to the residual pathway (result of layer normalization) **before** adding it back to the original **inputs**. This ensures that the residual connections are additive and do not interfere with the normalization process, facilitating better gradient flow and optimization stability.

Therefore, we end up with a code like this:
```python
# Transformer Block
class Block(torch.nn.Module):
    def __init__(self, configuration):
        super().__init__()
        self.layerNormalization1 = torch.nn.LayerNorm(configuration.numberOfEmbeddingDimensions)
        self.attention = CausalSelfAttention(configuration)
        self.layerNormalization2 = torch.nn.LayerNorm(configuration.numberOfEmbeddingDimensions)
        self.multiLayerPerceptron = MultiLayerPerceptron(configuration)
    
    def forward(self, inputs):
        inputs = inputs + self.attention(self.layerNormalization1(inputs))
        inputs = inputs + self.multiLayerPerceptron(self.layerNormalization1(inputs))
```

Here's how the residual pathways are structured:
- **Attention Layer**: After applying `self.layerNormalization1`, the residual (inputs) are added to `attention_output`. This adheres to the clean residual pathway because the normalization (`layerNormalization1`) is applied **before** adding to inputs.
- **MLP Layer**: Similarly, after applying `self.layerNormalization2`, the residual (inputs) is added to `mlp_output`. This also adheres to the clean residual pathway for the same reason as above.

And one more thing that is interesting to note is that, **Attention** is a **communication operation**, and it is where all the tokens line-up in a sequence and this is where the tokens communicate and exchange information. And **Attention** is an **aggregation function**, it's a **pooling function**, it's a **weighted sum function**, it's a **reduce operation**.

Whereas, **Multi Layer Perceptron** happens at every single token individually (**mapped**), and there is no information being exchanged or collected between the tokens.

So, the **Attention** is the **reduce** and **Multi Layer Perceptron** is the **map**. And what we end up with is a repeated application of **Map-Reduce**. And this is where the `tokens` communicate and this is where they *think* individually about the information that they gathered. And every one of these blocks, iteratively refines the representation inside the residual stream...

And now we can move on the to implementation of `CausalSelfAttention` and `MultiLayerPerceptron`...

### `MultiLayerPerceptron`

Let's now move on to the `MultiLayerPerceptron (MLP)`, and I implemented the class as follows...

```python
# Multi Layer Perceptron (MLP)
class MultiLayerPerceptron(torch.nn.Module):
    def __init__(self, configuration):
        super().__init__()
        
    def forward(self, inputs):
        pass
```

It is relatively straight forward. For now, we just have two `Linear` layers which wrap around a `GELU` non-linearity layer. So our block now becomes something like this:
```python
# Multi Layer Perceptron (MLP)
class MultiLayerPerceptron(torch.nn.Module):
    def __init__(self, configuration):
        super().__init__()
        self.currentFullyConnected = torch.nn.Linear(configuration.numberOfEmbeddingDimensions, 4 * configuration.numberOfEmbeddingDimensions)
        self.gelu = torch.nn.GELU(approximate="tanh")
        self.currentProjection = torch.nn.Linear(4 * configuration.numberOfEmbeddingDimensions, configuration.numberOfEmbeddingDimensions)

    def forward(self, inputs):
        inputs = self.currentFullyConnected(inputs)
        inputs = self.gelu(inputs)
        inputs = self.currentProjection(inputs)
        return inputs
```

Now when we swing over to the <a href="https://pytorch.org/docs/stable/generated/torch.nn.GELU.html">GELU PyTorch Documentation</a>, we see **two** different `GELU`(s) being hinted there:
1. Original GELU formulation (We will discuss this in a bit):
   $$ \text{GELU}(x) = x \cdot \Phi(x) $$
2. Approximate GELU formulation:
   $$ \text{GELU}(x) = 0.5 \cdot x \cdot \left(1 + \tanh \left( \frac{2}{\pi} \cdot \left(x + 0.044715 \cdot x^3 \right) \right) \right) $$

![GELU](ExplanationMedia/Images/GELU.png)

Just as a preview, we can see that `GELU` is basically like a `ReLU`, except there's **no exactly flat tail at exactly `0`**. Otherwise, it just looks more like a slightly *smoother* `ReLU`. And it comes from this paper <a href="https://arxiv.org/abs/1606.08415">"Gaussian Error Linear Units (GELUs)"</a> and there's a little bit of history here and I also invite you to step through the paper if you'd like. But for now, we will use the **approximate** version of the `GELU`, because that's what `GPT-2` in their model used...

Now, one other reason of why we prefer to use `GELU` is that, in previous notebooks we have spoken about the **Dead-ReLU-Neuron-Problem** where, in the tail of a `ReLU`, where it's exactly flat at `0`, any activations that fall there will get exactly `0` gradient (meaning that there's no change, there's no adaptation, there's no development of the network), but `GELU` always contributes to a **local-gradient** and so there's always going to be a change and there's always going to be an adaptation in a *smoothed-out* way which empirically working better, as demonstrated in the paper.

And we also followed the rule of *"Position-wise Feed-Forward Networks"* section of the original "Attention-is-all-you-need" paper, which is why we have the `4 * numberOfEmbeddingDimensions` in the shapes...

And finally we can now move on to implement the `CausalSelfAttention` part of the code...

### `CausalSelfAttention`

We can now start implementing our `CausalSelfAttention` block which is none other than the combination of **Scaled Dot-Product Attention** and **Multi-Head Attention**...

For now we have a skeleton like this:
```python
# Causal Self Attention (Scaled Dot-Product Attention + Multi-Head Attention)
class CausalSelfAttention(torch.nn.Module):
    def __init__(self, configuration):
        super().__init__()
    
    def forward(self, inputs):
        pass
```

Remember from our <a href="https://github.com/AvishakeAdhikary/Neural-Networks-From-Scratch/blob/main/GPT%20from%20Scratch.ipynb">GPT from Scratch notebook</a>, that **Multi-Head Attention** is just multiple **Scaled Dot-Product Attention**(s) running in parellel and their outputs are just being concatenated and that becomes the output.

Instead, we do a bunch of tensor *gymnastics* of mathematical operations of the same logic used behind both these **Multi-Head Attention** & **Scaled Dot-Product Attention** modules in a single block. But fundamentally, and algorithmically, nothing is different from what we implemented previously...

And this is what we end up with:
```python
# Causal Self Attention (Scaled Dot-Product Attention + Multi-Head Attention)
class CausalSelfAttention(torch.nn.Module):
    def __init__(self, configuration):
        super().__init__()
        assert configuration.numberOfEmbeddingDimensions % configuration.numberOfHeads == 0
        self.causalAttention = torch.nn.Linear(configuration.numberOfEmbeddingDimensions, 3 * configuration.numberOfEmbeddingDimensions)
        self.causalProjection = torch.nn.Linear(configuration.numberOfEmbeddingDimensions, configuration.numberOfEmbeddingDimensions)
        self.numberOfHeads = configuration.numberOfHeads
        self.numberOfEmbeddingDimensions = configuration.numberOfEmbeddingDimensions
        self.register_buffer("bias", torch.tril(torch.ones(configuration.blockSize, configuration.blockSize)).view(1, 1, configuration.blockSize, configuration.blockSize))
    
    def forward(self, inputs):
        B, T, C = inputs.size()
        query_key_value = self.causalAttention(inputs)
        query, key, value = query_key_value.split(self.numberOfEmbeddingDimensions, dim=2)
        query = query.view(B, T, self.numberOfHeads, C // self.numberOfHeads).transpose(1, 2)
        key = key.view(B, T, self.numberOfHeads, C // self.numberOfHeads).transpose(1, 2)
        value = value.view(B, T, self.numberOfHeads, C // self.numberOfHeads).transpose(1, 2)
        attention = (query @ key.transpose(-2, -1)) * (1.0 / math.sqrt(key.size(-1)))
        attention = attention.masked_fill(self.bias[:,:,:T,:T] == 0, float('-inf'))
        attention = F.softmax(attention, dim=-1)
        outputs = attention @ value
        outputs = outputs.transpose(1, 2).contiguous().view(B, T, C)
        outputs = self.causalProjection(outputs)
        return outputs
```

## GPT-2 Weights Transfer

Now that we have the skeleton ready, we can now move on to transfer the weights of the `Hugging Face GPT-2` to our `Custom GPT-2`...

### `from_pretrained()` - Skeleton

Let's start by understanding what we want to do first...

We want to have a `from_pretrained()` method in our `GPTModel` class, that will transfer the weights for any kind of model we pass it (among the `4` models that are there in `GPT-2`), and copy the weights of each of those parameters and ensure their sizes and shapes match perfectly...

And we also want our `from_pretrained()` method to be decorated by a `@classmethod` such that it could be accessed directly using the class reference and it is also able to modify the state of the class and return the appropriate model along with their appropriate parameters...

So for now we can have a skeleton like this:
```python
# GPT model architecture
class GPTModel(torch.nn.Module):
    def __init__(self, configuration):
        super().__init__()
        ... # Rest of the code

    # Method to transfer weights from Hugging Face GPT-2
    @classmethod
    def from_pretrained(cls, modelType):
        assert modelType in {'gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl'}
        from transformers import GPT2LMHeadModel
        print("Loading weights from pretrained gpt: %s" % modelType)
```

### `from_pretrained()` - Configuration

We can now create the separate configuartion arguements for all `4` `GPT-2` configurations...

And our code will look like this:
```python
# GPT model architecture
class GPTModel(torch.nn.Module):
    def __init__(self, configuration):
        super().__init__()
        ... # Rest of the code

    # Method to transfer weights from Hugging Face GPT-2
    @classmethod
    def from_pretrained(cls, modelType):
        assert modelType in {'gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl'}
        from transformers import GPT2LMHeadModel
        print("Loading weights from pretrained gpt: %s" % modelType)

        # Creating separate configurations for separate GPT-2 models
        configurationArguements = {
            'gpt2':         dict(numberOfLayers=12, numberOfHeads=12, numberOfEmbeddingDimensions=768),  # 124M parameters
            'gpt2-medium':  dict(numberOfLayers=24, numberOfHeads=16, numberOfEmbeddingDimensions=1024), # 350M parameters
            'gpt2-large':   dict(numberOfLayers=36, numberOfHeads=20, numberOfEmbeddingDimensions=1280), # 774M parameters
            'gpt2-xl':      dict(numberOfLayers=48, numberOfHeads=25, numberOfEmbeddingDimensions=1600), # 1558M parameters
        }[modelType]
        configurationArguements['vocabularySize'] = 50257
        configurationArguements['blockSize'] = 1024
```

Now we can unpack our configurations into a variable called `configuration` based on the `modelType` arguement we pass as a parameter. And then we can initialize our `GPTModel` based on the `configuration` that we initialize. And then we can also copy the state-dictionary containing all the layers in our model in a variable called `stateDictionary` with the method `state_dict()` and unpack it's keys using the `keys()` method into a variable called `stateDictionaryKeys` (We have to keep in mind that we discard all the buffers that are not a part of the parameters like **Attention Mask** and **Attention Bias**)...

So, now we have a code like this:
```python
# GPT model architecture
class GPTModel(torch.nn.Module):
    def __init__(self, configuration):
        super().__init__()
        ... # Rest of the code

    # Method to transfer weights from Hugging Face GPT-2
    @classmethod
    def from_pretrained(cls, modelType):
        assert modelType in {'gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl'}
        from transformers import GPT2LMHeadModel
        print("Loading weights from pretrained gpt: %s" % model_type)

        # Creating separate configurations for separate GPT-2 models
        configurationArguements = {
            'gpt2':         dict(numberOfLayers=12, numberOfHeads=12, numberOfEmbeddingDimensions=768),  # 124M parameters
            'gpt2-medium':  dict(numberOfLayers=24, numberOfHeads=16, numberOfEmbeddingDimensions=1024), # 350M parameters
            'gpt2-large':   dict(numberOfLayers=36, numberOfHeads=20, numberOfEmbeddingDimensions=1280), # 774M parameters
            'gpt2-xl':      dict(numberOfLayers=48, numberOfHeads=25, numberOfEmbeddingDimensions=1600), # 1558M parameters
        }[modelType]
        configurationArguements['vocabularySize'] = 50257
        configurationArguements['blockSize'] = 1024

        configuration = GPTConfiguration(**configurationArguements)
        model = GPTModel(configuration)

        stateDictionary = model.state_dict()
        stateDictionaryKeys = stateDictionary.keys()
        stateDictionaryKeys = [key for key in stateDictionaryKeys if not key.endswith('.attention.bias')]

        return model
```

### `from_pretrained()` - Hugging Face Initialization

Now that we have our own custom model completely initialized, we can go ahead and initialize the `Hugging Face GPT-2 Model` into a single variable and it's state dictionary and keys into other variables called `huggingfaceStateDictionary` and `huggingfaceStateDictionaryKeys`...

And then we can start copying the weights after **ignoring the buffers**. But now, before copying we have to keep in mind that the original code for the `GPT-2` model was trained using the `TensorFlow` library and some of the weights are **transposed** in that architecture, so we will manually hard-code those weights and **copy them after transposing them to their original PyTorch form**...

One last thing to be careful about is, in our model we are using **custom names for our variables** but the `Hugging Face GPT-2 Model` has an architecture of **short forms**, so it is better to have a `parameterKeyMapping` that **maps our custom keys with the Hugging Face GPT-2 keys of the state-dictionary** such that it becomes much easier to iterate through...

So, now we have a final code like this:
```python
# GPT model architecture
class GPTModel(torch.nn.Module):
    def __init__(self, configuration):
        super().__init__()
        ... # Rest of the code

    # Method to transfer weights from Hugging Face GPT-2
    @classmethod
    def from_pretrained(cls, modelType):
        assert modelType in {'gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl'}
        from transformers import GPT2LMHeadModel
        print("Loading weights from pretrained gpt: %s" % modelType)
        
        # Creating separate configurations for separate GPT-2 models
        blockSize = 1024
        vocabularySize = 50257
        configurationArguements = {
            'gpt2':         dict(numberOfLayers=12, numberOfHeads=12, numberOfEmbeddingDimensions=768),  # 124M parameters
            'gpt2-medium':  dict(numberOfLayers=24, numberOfHeads=16, numberOfEmbeddingDimensions=1024), # 350M parameters
            'gpt2-large':   dict(numberOfLayers=36, numberOfHeads=20, numberOfEmbeddingDimensions=1280), # 774M parameters
            'gpt2-xl':      dict(numberOfLayers=48, numberOfHeads=25, numberOfEmbeddingDimensions=1600), # 1558M parameters
        }[modelType]
        configurationArguements['vocabularySize'] = 50257
        configurationArguements['blockSize'] = 1024

        configuration = GPTConfiguration(**configurationArguements)
        model = GPTModel(configuration)
        stateDictionary = model.state_dict()
        stateDictionaryKeys = stateDictionary.keys()
        stateDictionaryKeys = [key for key in stateDictionaryKeys if not key.endswith('.attention.bias')]

        huggingfaceModel = GPT2LMHeadModel.from_pretrained(modelType)
        huggingfaceStateDictionary = huggingfaceModel.state_dict()
        huggingfaceStateDictionaryKeys = huggingfaceStateDictionary.keys()
        huggingfaceStateDictionaryKeys = [key for key in huggingfaceStateDictionaryKeys if not key.endswith('.attn.masked_bias')]
        huggingfaceStateDictionaryKeys = [key for key in huggingfaceStateDictionaryKeys if not key.endswith('.attn.bias')]
        transposedParameters = ['attn.c_attn.weight', 'attn.c_proj.weight', 'mlp.c_fc.weight', 'mlp.c_proj.weight']

        assert len(huggingfaceStateDictionaryKeys) == len(stateDictionaryKeys), f"Mismatched Keys: {len(huggingfaceStateDictionaryKeys)} != {len(stateDictionaryKeys)}"

        parameterKeyMapping = {
            customKey: huggingfaceKey
            for customKey, huggingfaceKey in zip(stateDictionaryKeys, huggingfaceStateDictionaryKeys)
            }

        for customKey, huggingfaceKey in parameterKeyMapping.items():
            if (huggingfaceStateDictionary[huggingfaceKey].shape != stateDictionary[customKey].shape):
                # Special treatment for the Conv1D weights (Transposed Weights)
                if (huggingfaceKey.endswith(word) for word in transposedParameters):
                    assert huggingfaceStateDictionary[huggingfaceKey].shape[::-1] == stateDictionary[customKey].shape
                    with torch.no_grad():
                        stateDictionary[customKey].copy_(huggingfaceStateDictionary[huggingfaceKey].t())
                # Vanilla copy for other parameters
                else:
                    assert huggingfaceStateDictionary[huggingfaceKey].shape == stateDictionary[customKey].shape
                    with torch.no_grad():
                        stateDictionary[customKey].copy_(huggingfaceStateDictionary[huggingfaceKey])
        
        return model
```

As soon as we do this, you will see that it will start downloading the model from Hugging Face like this:
```bash
Loading weights from pretrained gpt: gpt2
config.json: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 665/665 [00:00<?, ?B/s]
model.safetensors:  34%|████████████████████████████████████████████████████▋                                                                                                    | 189M/548M [00:18<00:35, 10.1MB/s]
```

Once it's done we can now move on to the generation phase of the model...